Definiamo il tipo delle espressioni aritmetiche.

In [1]:
type aexp =
  | Num of int            (* number literal *)
  | Plus of aexp * aexp   (* sum *)
  | Neg of aexp           (* unary negation *)
  | Times of aexp * aexp  (* product *)
  | Div of aexp * aexp    (* division *)
  ;;

type aexp =
    Num of int
  | Plus of aexp * aexp
  | Neg of aexp
  | Times of aexp * aexp
  | Div of aexp * aexp


Vogliamo implementare un valutatore di espressioni aritmetiche con gestione "safe" degli errori. Definiamo il tipo di ritorno del valutatore come

In [2]:
type result =
  | Right of int
  | Error of string;;

type result = Right of int | Error of string


Un valore di tipo `Right` rappresenta un'esecuzione corretta, mentre un `Error` indica che qualcosa è andato storto durante la valutazione.

Scrivere la funzione `eval` con questo effetto. Per ora, l'unico errore possibile è una divisione per zero.

In [3]:
let controlla_errori (f: int -> int -> int) (op1: result) (op2: result) : result =
  match op1, op2 with
    | Right n1, Right n2 -> Right (f n1 n2)
    | Error s1, Right _ -> Error ("nel primo addendo: " ^ s1)
    | Right _, Error s2 -> Error ("nel secondo addendo: " ^ s2)
    | Error s1, Error s2 -> Error ("tutto sbagliato: " ^ s1 ^ " " ^ s2);;

let rec eval (e: aexp) : result =
  match e with
    | Num n -> Right n
    | Plus (e1, e2) -> controlla_errori (+) (eval e1) (eval e2)
    | Neg e1 -> (match eval e1 with
      | Right n1 -> Right (-n1)
      | Error s1 -> Error s1
      )
    | Times (e1, e2) -> controlla_errori (fun a b -> a * b) (eval e1) (eval e2)
    | Div (e1, e2) -> (match eval e1, eval e2 with
      | Right n1, Right 0 -> Error "divisione per zero"
      | op1, op2 -> controlla_errori (/) op1 op2);;

val controlla_errori : (int -> int -> int) -> result -> result -> result =
  <fun>


val eval : aexp -> result = <fun>


In [4]:
let test = Plus (Num 10, Div (Num 10, Num 0));;  (* 10 + (10 / 0) *)
let test2 = Div (Plus (Num 10, Num 10), Num 0);; (* (10 + 10) / 0 *)
eval test;;
eval test2;;

val test : aexp = Plus (Num 10, Div (Num 10, Num 0))


val test2 : aexp = Div (Plus (Num 10, Num 10), Num 0)


- : result = Error "nel secondo addendo: divisione per zero"


- : result = Error "divisione per zero"


Vogliamo estendere il comportamento in caso di errore in modo che riporti alcune informazioni aggiuntive. Definiamo un record per gli errori invece di una semplice stringa: vogliamo anche sapere l'espressione che ha causato l'errore, e il valore calcolato per i suoi parametri.

In [5]:
type error = {
    msg: string;      (* Il messaggio di errore *)
    expr: aexp;       (* L'espressione che ha causato l'errore *)
    args: int list;   (* Valori dei sottoalberi dell'espressione *)
};;

type result =
  | Right of int
  | Error of error;;  (* non è più string *)

type error = { msg : string; expr : aexp; args : int list; }


type result = Right of int | Error of error


Per esempio, se l'espressione di errore è
```ocaml
Div ((Neg (Num 3)), (Times (Num 10, Num 0)))
(* (-3) / (10 * 0) *)
```
il corrispondente record di errore dev'essere
```ocaml
{ msg = "Division by zero"; expr = Div ((Neg (Num 3)), (Times (Num 10, Num 0))); args = [-3; 0] }
```
dato che gli "argomenti" della divisione sono `-3` e `0`.

Aggiornare la funzione `eval` per calcolare l'errore strutturato

In [6]:
let controlla_errori (f: int -> int -> int) (op1: result) (op2: result) : result =
  match op1, op2 with
    | Right n1, Right n2 -> Right (f n1 n2)
    | Error err1, _ -> Error err1
    | Right _, Error err2 -> Error err2;;

let rec eval (e: aexp) : result =
  match e with
    | Num n -> Right n
    | Plus (e1, e2) -> controlla_errori (+) (eval e1) (eval e2)
    | Neg e1 -> (match eval e1 with
      | Right n1 -> Right (-n1)
      | Error err1 -> Error err1
      )
    | Times (e1, e2) -> controlla_errori (fun a b -> a * b) (eval e1) (eval e2)
    | Div (e1, e2) -> (match eval e1, eval e2 with
      | Right n1, Right 0 -> Error {
          msg="divisione per zero";
          expr=Div (e1, e2);
          args=[n1; 0]
        }
      | op1, op2 -> controlla_errori (/) op1 op2);;

val controlla_errori : (int -> int -> int) -> result -> result -> result =
  <fun>


val eval : aexp -> result = <fun>


In [7]:
let test = Div ((Neg (Num 3)), (Times (Num 10, Num 0)));; (* (-3) / (10 * 0) *)
let test2 = Neg (test);; (* -((-3) / (10 * 0)) *)

eval test;;
eval test2;;

val test : aexp = Div (Neg (Num 3), Times (Num 10, Num 0))


val test2 : aexp = Neg (Div (Neg (Num 3), Times (Num 10, Num 0)))


- : result =
Error
 {msg = "divisione per zero";
  expr = Div (Neg (Num 3), Times (Num 10, Num 0)); args = [-3; 0]}


- : result =
Error
 {msg = "divisione per zero";
  expr = Div (Neg (Num 3), Times (Num 10, Num 0)); args = [-3; 0]}


Definiamo ora una funzione `print_error` per stampare l'errore: deve produrre una stringa che contenga il messaggio di errore, il tipo di operazione che ha causato l'errore (indicato nel nodo root di `expr`) e i suoi parametri. Per esempio, per l'errore di prima si deve stampare
"Division by zero: applying Div to 3 and 0"

In [8]:
let rec print_aexp (e: aexp) : string =
  match e with
    | Num n -> string_of_int n
    | Plus (e1, e2) -> "(" ^ print_aexp e1 ^ " + " ^ print_aexp e2 ^ ")"
    | Neg e1 -> "-" ^ print_aexp e1 ^ ""
    | Times (e1, e2) -> "(" ^ print_aexp e1 ^ " * " ^ print_aexp e2 ^ ")"
    | Div (e1, e2) -> "(" ^ print_aexp e1 ^ " / " ^ print_aexp e2 ^ ")";;

let print_list (l: int list) : string =
  List.fold_left (fun acc elem -> acc ^ elem ^ ", ") "" (List.map string_of_int l)

let print_error (err: error) : string =
  err.msg ^ " nell'espressione " ^ print_aexp err.expr ^ ". Le sottoespressioni valutano a " ^ print_list err.args;;

val print_aexp : aexp -> string = <fun>


val print_list : int list -> string = <fun>


val print_error : error -> string = <fun>


In [9]:
let test3 = Plus (Num 4, test);;

print_aexp test3;;

(* Serve questo match perché (eval test3) è di tipo result, e l'interprete non sa
 * che in realtà è un Error err
 *)
match eval test3 with
  | Error err -> print_error err
  | Right _ -> "";;

val test3 : aexp = Plus (Num 4, Div (Neg (Num 3), Times (Num 10, Num 0)))


- : string = "(4 + (-3 / (10 * 0)))"


- : string =
"divisione per zero nell'espressione (-3 / (10 * 0)). Le sottoespressioni valutano a -3, 0, "


Aggiungere alla funzione `eval` un minimo e massimo valore per gli interi (costanti fissate). Se in qualunque momento la valutazione di espressione produce un valore fuori dai due estremi, bisogna invece ritornare un appropriato errore.

In [10]:
let maxint = 100;;
let minint = -100;;

val maxint : int = 100


val minint : int = -100
